# Basic workflow with ManGO

## Authentication with iCommands

<div class="alert alert-block alert-info">
<h3>Every seven days</h3>
    
1. Go to https://mango.kuleuven.be/
2. In the tab of your zone, click on "How to connect"
3. Copy the snippet provided under "iCommands Client on Linux".

<font size=3>Then **paste the snippet** in the cell below, right under `%%bash`, like the (anonimized) example below.</font>

(Replacing the `USERNAME` with your username and the `TOKEN` with the password provided by "How to connect" should also work.)

**You don't need to do this every time: The authentication lasts 7 days.**
</div>

In [ ]:
%%bash
mkdir -p ~/.irods
cat > ~/.irods/irods_environment.json <<'EOF'
{
    "irods_host": "vsc.irods.hpc.kuleuven.be",
    "irods_port": 1247,
    "irods_zone_name": "vsc",
    "irods_authentication_scheme": "pam_password",
    "irods_encryption_algorithm": "AES-256-CBC",
    "irods_encryption_salt_size": 8,
    "irods_encryption_key_size": 32,
    "irods_encryption_num_hash_rounds": 8,
    "irods_user_name": "USERNAME",
    "irods_ssl_ca_certificate_file": "",
    "irods_ssl_verify_server": "cert",
    "irods_client_server_negotiation": "request_server_negotiation",
    "irods_client_server_policy": "CS_NEG_REQUIRE",
    "irods_default_resource": "default",
    "irods_cwd": "/vsc/home"
}
EOF
iinit -h | grep Version | grep -v -q 4.2. || sed -i 's/"irods_authentication_scheme": "pam_password"/"irods_authentication_scheme": "PAM"/' ~/.irods/irods_environment.json
echo 'TOKEN' | iinit --ttl 168 >/dev/null && echo You are now authenticated to irods. Your session is valid for 168 hours.
ils

## Setting up

Set up ManGO and load any other libraries you need.

In [2]:
import os
import ssl
from irods.session import iRODSSession
from irods.models import Collection, DataObject, DataObjectMeta
from irods.column import Criterion
from mango_mdschema import Schema
import logging
logger = logging.getLogger("mango_mdschema") # to read the validation
logger.setLevel(logging.INFO)
try:
    env_file = os.environ['IRODS_ENVIRONMENT_FILE']
except KeyError:
    env_file = os.path.expanduser('~/.irods/irods_environment.json')

ssl_context = ssl.create_default_context(
        purpose=ssl.Purpose.SERVER_AUTH,
        cafile=None, capath=None, cadata=None
        )
ssl_settings = {'ssl_context': ssl_context}

Since we are working interactively we will create an `irods.session.iRODSSession` object and then close it at the end of the notebook with `session.cleanup()`. If you were working on a script, you could run all your code inside a `with` statement.

In [3]:
session = iRODSSession(irods_env_file=env_file, **ssl_settings)

In [ ]:
home_dir = "/vsc/home/ManGO-VIB/"

## WRITE YOUR CODE BELOW THIS
In all cases, you may use more cells than are provided

Below, use your VSC number (e.g. "vsc30000") to provide the right path for the collection where you will store your output as well as the name (not full path) of the vcf file you created.

In [ ]:
output_dir = home_dir + "output/" + "USER/" # it might be a different name
output_filename = "THE NAME OF YOUR OUTPUT FILE"

In the cell below, query for files annotated with the `fastq` Schema and for which the "organism" is "human". Then retrieve the path after the query, (check the checksums) and download the files to your location.

Once you have run your analysis, use the cell below to upload your output.

Download the metadata schema from the home directory ("vcf-1.0.0-published.json") and inspect the fields it requests.

In [ ]:
schema_filename = "vcf-1.0.0-published.json"
# DOWNLOAD THE FILE FROM MANGO

In [ ]:
vcf_schema = Schema(schema_filename)
print(vcf_schema)

Create a dictionary with the metadata you know that you could use to fill the VCF schema.
Note that "sample_id" should be a LIST with the values of `mgs.fastq.sample.sample_id` from your input files. So:
- Retrieve the sample_id and organism from the input files
- Register the software used to generate the VCF file.

Now that you have your dictionary, validate it and add the metadata to your uploaded output file.

## CLEAN UP 
<div class="alert alert-block alert-warning">
    <font size=4><b>Do not forget to clean up your session!</b></font>
</div>

In [ ]:
# leave this cell at the end and running every time you are done
session.cleanup()